## 절차

1. 연구목표 설정
2. 데이터 수집
 - https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true
 - https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true
 - selenium 이용
 - 페이지 로드되면, end를 작동시켜 밑으로 내린다
 - 리뷰 수집
  > 이름  
  > 별점  
  > 등록일  
  > 내용  
  > 좋아요  
3. 데이터 준비(정제, 변환, 통합)
  > 정규식으로 필터링(욕설, 기타 표현에 대한 정책)  
  > 파생변수 → 형태소, 점수 지표를 이용한 계산, 사전에 없는 표현에 대한 유사어 검사(확장성) 등
4. 데이터 탐색(시각화, 통찰 과정)
  > 워드클라우드, 바차트, 히스토그램, 산포도
5. 모델링
6. 시스템 통합(사전의 재구축(선순환 구조) 및 반영, 서비스)

In [1]:
from selenium import webdriver as wd

In [11]:
import urllib
import time

In [8]:
target_urls = [
'https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true',
'https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true'
]

In [19]:
# 웹 드라이버
driver = wd.Chrome('../tool/chromedriver.exe')

In [21]:
# selenium.webdriver.common.keys.Keys
from selenium.webdriver.common.keys import Keys

# 타겟 사이트 데이터 수집
def getData(app_name) :
    # 스크롤바를 밑으로 내린다(15번)
    for n in range(15) :
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        # ajax를 통해서 리뷰가 로드될 것이다
        time.sleep(3)
    
    reviews = driver.find_elements_by_class_name('d15Mdf')
    tmp = []
    for review in reviews :
        # 이름
        name = review.find_element_by_class_name('X43Kjb').text
        # 별점
        star = review.find_element_by_class_name('pf5lIe>div').get_attribute('aria-label')
        # 등록일
        regi = review.find_element_by_class_name('p2TkOb').text
        # 내용
        content = review.find_element_by_class_name('UD7Dzf>span').text
        # 좋아요
        good = review.find_element_by_class_name('jUL89d.y92BAb').text
        dic = {
            'name' : name,
            'star' : star,
            'regi' : regi,
            'content' : content,
            'good' : good,
            'appname' : app_name
        }
        tmp.append(dic)
    # 예외처리 생략
    return tmp


In [22]:
outputs = list()
app_names = ['네이버웹툰', '다음웹툰']
for idx, target_url in enumerate(target_urls) :
    # print (idx, target_url)
    
    # 사이트 접속
    driver.get(target_url)
    outputs.extend(getData(app_names[idx]))
    time.sleep(5)

In [ ]:
outputs

In [24]:
len(outputs)

80

In [25]:
import pandas as pd
import numpy as np

###### pd.DataFrame.from_dict()

In [26]:
df = pd.DataFrame.from_dict(outputs)
df.head()

,appname,content,good,name,regi,star
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019년 8월 17일,별표 5개 만점에 1개를 받았습니다.
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019년 8월 20일,별표 5개 만점에 1개를 받았습니다.
2,네이버웹툰,"""개를 낳았다""를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸...",81,롄쀼,2019년 8월 14일,별표 5개 만점에 5개를 받았습니다.
3,네이버웹툰,"한 두 번도 아니고 와이파이, 데이터 둘 다 네이버웹툰만 켜면 제일을 왜 못합니까?...",1,초보사리엘,2019년 8월 20일,별표 5개 만점에 1개를 받았습니다.
4,네이버웹툰,아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안...,93,tv부기,2019년 8월 11일,별표 5개 만점에 1개를 받았습니다.


In [27]:
import re

In [30]:
# 'star'에서 점수만 추출
sample = '별표 5개 만점에 3개를 받았습니다.'
# p = re.compile('\d+')
p = re.compile('[0-9]+')
print(p.findall(sample)[1])

3


In [31]:
df['star'] = df['star'].apply(lambda x : p.findall(x)[1])

In [32]:
df.head(2)

,appname,content,good,name,regi,star
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019년 8월 17일,1
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019년 8월 20일,1


In [33]:
df['star'].unique()

array(['1', '5', '2', '3', '4'], dtype=object)

In [34]:
# 'regi'에서 날짜를 yyyy-mm-dd 형태로 날짜 포맷팅
sample = '2019년 8월 16일'
p = re.compile('(\d+)\w\s(\d+)\w\s(\d+)\w')
p.sub('\g<1>-\g<2>-\g<3>', sample)

'2019-8-16'

In [36]:
import datetime

In [37]:
datetime.datetime.strptime('2019-8-16', '%Y-%m-%d').date()

datetime.date(2019, 8, 16)

In [39]:
df['regi'] = df['regi'].apply(lambda x : datetime.datetime.strptime(p.sub('\g<1>-\g<2>-\g<3>', x), '%Y-%m-%d').date())

In [40]:
df.head()

,appname,content,good,name,regi,star
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019-08-17,1
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019-08-20,1
2,네이버웹툰,"""개를 낳았다""를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸...",81,롄쀼,2019-08-14,5
3,네이버웹툰,"한 두 번도 아니고 와이파이, 데이터 둘 다 네이버웹툰만 켜면 제일을 왜 못합니까?...",1,초보사리엘,2019-08-20,1
4,네이버웹툰,아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안...,93,tv부기,2019-08-11,1


In [41]:
# 백업
df.to_csv('./data/app_review_mid.xls', encoding='utf-8')

In [ ]:
df['content']

In [ ]:
# 명사, 형용사 기준으로 하고, 동사는 확장성/가능성을 두고 체크
# 형태소 분석을 하기 전에 사전에 포함되어 있지 않는 문자들을 제거

In [45]:
# 1. 영어, 숫자 제거 ↔ 한글과 특수기호, whitespace 추출
p = re.compile('[a-zA-Z0-9]+')
df['content'].apply(lambda x : p.sub('', x))

0     쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...
1     저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...
2     "개를 낳았다"를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸...
3     한 두 번도 아니고 와이파이, 데이터 둘 다 네이버웹툰만 켜면 제일을 왜 못합니까?...
4     아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키개?절대안줌 내...
5     자기의 취향에 걸맞는 것도 찾기가 쉽고 또한 어느것이 도 인기가 도 많은지 바로바로...
6     아니 왜 무료쿠키 안줘요? 앱깔고 회원가입하면 쿠키 개 준다길래 앱깔고 회원가입도 ...
7     안녕하세요. 제가 할거가 없어서 찾고 있었는데 딱 이게 발견 했습니다. 그래서 한번...
8     요즘 자꾸 모자이크칸이 많이뜨고 이미지가 안나오네요. 크롬으로 네이버직좁 들어가서 ...
9     진짜 진짜 재밌게 보고 있어요 !! 💗 근데 와이파이 없이 볼수는 없나요 ? ㅜ 고...
10    저기요 돼지만화는 언제 자르실건지.....? 그거 보면 진심 화나요..... 조선팔...
11    네이버 웹툰은 컷마다 볼 수 있는 댓글과 보기편한 점이 있어서 좋지만 다른 새로운 ...
12    결제는 네이버 어플에서 바로하게하고 미리볼려면 다른 어플(네이버웹툰어플)을 깔아야한...
13    제가 계정을 만들려고 아이디 비밀번호 만들고 전화번호도 쳤고 인증번호가 안오는것 같...
14    저만 그런건지는 모르겠는데 웹툰 밑으로 스크롤해서 보다가 중간에 만화 장면이 나타나...
15    저 의견사항에서 얘기 했는데 답장이 하도 안와서 여기서 말합니다 제 오해 일 수 있...
16    물론 아주 편리한 앱입니다 하지만 하나하나 이렇게 부모님 인증에 상관없는 거의 모든...
17    와....이거 무료쿠키 받을려고 다운로드 했는데 쿠키를 안줌.. 그거 다운로

In [ ]:
# 컬럼 'sentiment' 추가
# star 컬럼을 참조하여
# pos(긍정) : 별점기준 4 or 5
# neg(부정) : 별점기준 1 or 2 or 3

In [46]:
df.dtypes

appname    object
content    object
good       object
name       object
regi       object
star       object
dtype: object

In [49]:
df['star'] = df['star'].astype(int)

In [50]:
df.dtypes

appname    object
content    object
good       object
name       object
regi       object
star        int32
dtype: object

In [52]:
# 방법 1
tmp = []
for star in df['star'] :
    if star>3 :
        tmp.append('pos')
    else :
        tmp.append('neg')
tmp[:5]

['neg', 'neg', 'pos', 'neg', 'neg']

In [ ]:
# 방법 2
# x>3 and 'pos' or 'neg'
# df['star'].apply(lambda x : x>3 and 'pos' or 'neg')

In [53]:
# 방법 3
def checkPosNeg(x) :
    if x>3 :
        return 'pos'
    return 'neg'

In [55]:
df['sentiment'] = df['star'].apply(checkPosNeg)

###### 마스크 인자로 조건이 참인 것을 체크 : .mask()

In [57]:
df['star'].mask(df['star']>3, 'pos')[:10]

0      1
1      1
2    pos
3      1
4      1
5    pos
6      2
7      3
8      1
9    pos
Name: star, dtype: object

###### 조건이 거짓인 데이터를 찾아서 Nan or 대체 : .where()

In [58]:
df['star'].where(df['star']>3, 'neg')[:10]

0    neg
1    neg
2      5
3    neg
4    neg
5      5
6    neg
7    neg
8    neg
9      4
Name: star, dtype: object

In [59]:
df.head(2)

,appname,content,good,name,regi,star,sentiment
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019-08-17,1,neg
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019-08-20,1,neg


In [ ]:
# 트위터 형태소 분석기의 분해 함수(품사 포함)를 이용하여
# 리뷰를 형태소로 분해한 내용을 새로운 컬럼 'classifier'에 추가한다

In [61]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [62]:
# 형태소 분석
tmp1 = pos_tagger.pos(df['content'][0])
tmp1

[('쿠키', 'Noun'),
 ('왜', 'Noun'),
 ('안주세요', 'Verb'),
 ('몇번', 'Noun'),
 ('째', 'Suffix'),
 ('야', 'Josa'),
 ('진짜', 'Noun'),
 ('무료', 'Noun'),
 ('쿠키', 'Noun'),
 ('~', 'Punctuation'),
 ('이러면서', 'Verb'),
 ('안주는게', 'Verb'),
 ('한두', 'Modifier'),
 ('번인', 'Noun'),
 ('가', 'Josa'),
 ('내', 'Noun'),
 ('가', 'Josa'),
 ('가입', 'Noun'),
 ('을', 'Josa'),
 ('얼마나', 'Noun'),
 ('많이', 'Adverb'),
 ('했는데', 'Verb'),
 ('방금', 'Noun'),
 ('알바', 'Noun'),
 ('가입', 'Noun'),
 ('머시', 'Noun'),
 ('했거든요', 'Verb'),
 ('?', 'Punctuation'),
 ('10', 'Number'),
 ('개', 'Noun'),
 ('빨리', 'Adverb'),
 ('주세요', 'Verb'),
 (';;', 'Punctuation'),
 ('진짜', 'Noun'),
 ('나', 'Noun'),
 ('말고도', 'Josa'),
 ('많더만', 'Adjective'),
 ('이', 'Determiner'),
 ('정도', 'Noun'),
 ('면', 'Josa'),
 ('네이버', 'Noun'),
 ('이', 'Determiner'),
 ('거', 'Noun'),
 ('고객', 'Noun'),
 ('기만', 'Noun'),
 ('이지', 'Josa'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 (';;;;', 'Punctuation'),
 ('그리고', 'Conjunction'),
 ('작가', 'Noun'),
 ('님', 'Suffix'),
 ('들', 'Verb'),
 ('다', 'Adverb'),
 ('좋거든요', 'Adje

In [64]:
len(tmp1)

91

###### 형태소 분석시 정규화 및 어근화 처리를 수행(선택 사항) : norm = True, stem = True

In [63]:
tmp2 = pos_tagger.pos(df['content'][0], norm = True, stem = True)
tmp2

[('쿠키', 'Noun'),
 ('왜', 'Noun'),
 ('알다', 'Verb'),
 ('몇번', 'Noun'),
 ('째', 'Suffix'),
 ('야', 'Josa'),
 ('진짜', 'Noun'),
 ('무료', 'Noun'),
 ('쿠키', 'Noun'),
 ('~', 'Punctuation'),
 ('이르다', 'Verb'),
 ('알다', 'Verb'),
 ('한두', 'Modifier'),
 ('번인', 'Noun'),
 ('가', 'Josa'),
 ('내', 'Noun'),
 ('가', 'Josa'),
 ('가입', 'Noun'),
 ('을', 'Josa'),
 ('얼마나', 'Noun'),
 ('많이', 'Adverb'),
 ('하다', 'Verb'),
 ('방금', 'Noun'),
 ('알바', 'Noun'),
 ('가입', 'Noun'),
 ('머시', 'Noun'),
 ('하다', 'Verb'),
 ('?', 'Punctuation'),
 ('10', 'Number'),
 ('개', 'Noun'),
 ('빨리', 'Adverb'),
 ('줄다', 'Verb'),
 (';;', 'Punctuation'),
 ('진짜', 'Noun'),
 ('나', 'Noun'),
 ('말고도', 'Josa'),
 ('많다', 'Adjective'),
 ('이', 'Determiner'),
 ('정도', 'Noun'),
 ('면', 'Josa'),
 ('네이버', 'Noun'),
 ('이', 'Determiner'),
 ('거', 'Noun'),
 ('고객', 'Noun'),
 ('기만', 'Noun'),
 ('이지', 'Josa'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 (';;;;', 'Punctuation'),
 ('그리고', 'Conjunction'),
 ('작가', 'Noun'),
 ('님', 'Suffix'),
 ('들다', 'Verb'),
 ('다', 'Adverb'),
 ('좋다', 'Adjective'),
 (

In [65]:
len(tmp2)

91

In [69]:
# 대상을 좁힌다 → 명사 : Noun, 형용사 : Adjective, 동사(예비) : Verb
# 위 품사들만 남기고 제거
tags = ['Noun', 'Adjective', 'Verb']
for word, tag in tmp2 :   # 튜플 형태이므로
    if tag in tags :
        print(word, tag)

쿠키 Noun
왜 Noun
알다 Verb
몇번 Noun
진짜 Noun
무료 Noun
쿠키 Noun
이르다 Verb
알다 Verb
번인 Noun
내 Noun
가입 Noun
얼마나 Noun
하다 Verb
방금 Noun
알바 Noun
가입 Noun
머시 Noun
하다 Verb
개 Noun
줄다 Verb
진짜 Noun
나 Noun
많다 Adjective
정도 Noun
네이버 Noun
거 Noun
고객 Noun
기만 Noun
장난 Noun
하나 Noun
작가 Noun
들다 Verb
좋다 Adjective
왜 Noun
자꾸 Noun
하위 Noun
짜르 Noun
세 Noun
장난 Noun
하나 Noun
작가 Noun
들다 Verb
작품 Noun
장난 Noun
이상하다 Adjective
만화 Noun
안 Noun
짜르 Noun
스토리 Noun
탄탄 Noun
작화 Noun
좋다 Adjective
하위 Noun
짤 Noun


In [71]:
[ tag for tag in tmp2 if tag[1] in tags ]
# [ (word, tag) for word, tag in tmp2 if tag in tags ]

[('쿠키', 'Noun'),
 ('왜', 'Noun'),
 ('알다', 'Verb'),
 ('몇번', 'Noun'),
 ('진짜', 'Noun'),
 ('무료', 'Noun'),
 ('쿠키', 'Noun'),
 ('이르다', 'Verb'),
 ('알다', 'Verb'),
 ('번인', 'Noun'),
 ('내', 'Noun'),
 ('가입', 'Noun'),
 ('얼마나', 'Noun'),
 ('하다', 'Verb'),
 ('방금', 'Noun'),
 ('알바', 'Noun'),
 ('가입', 'Noun'),
 ('머시', 'Noun'),
 ('하다', 'Verb'),
 ('개', 'Noun'),
 ('줄다', 'Verb'),
 ('진짜', 'Noun'),
 ('나', 'Noun'),
 ('많다', 'Adjective'),
 ('정도', 'Noun'),
 ('네이버', 'Noun'),
 ('거', 'Noun'),
 ('고객', 'Noun'),
 ('기만', 'Noun'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 ('작가', 'Noun'),
 ('들다', 'Verb'),
 ('좋다', 'Adjective'),
 ('왜', 'Noun'),
 ('자꾸', 'Noun'),
 ('하위', 'Noun'),
 ('짜르', 'Noun'),
 ('세', 'Noun'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 ('작가', 'Noun'),
 ('들다', 'Verb'),
 ('작품', 'Noun'),
 ('장난', 'Noun'),
 ('이상하다', 'Adjective'),
 ('만화', 'Noun'),
 ('안', 'Noun'),
 ('짜르', 'Noun'),
 ('스토리', 'Noun'),
 ('탄탄', 'Noun'),
 ('작화', 'Noun'),
 ('좋다', 'Adjective'),
 ('하위', 'Noun'),
 ('짤', 'Noun')]

In [72]:
df['classifier'] = df['content'].apply(lambda x : [ tag for tag in pos_tagger.pos(x, norm = True, stem = True) if tag[1] in tags ])

In [73]:
df['classifier'][:2]

0    [(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)...
1    [(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ...
Name: classifier, dtype: object

In [74]:
df.head(2)

,appname,content,good,name,regi,star,sentiment,classifier
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019-08-17,1,neg,"[(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)..."
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019-08-20,1,neg,"[(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ..."


In [75]:
import json

# 감정사전, 대략 7만여 개의 데이터에 긍정/부정지수 값이 세팅된 사전
with open('./data/SentiWord_info.json', encoding='utf-8') as f :
    score_dic = json.load(f)

In [ ]:
# 리뷰에 대한 감정사전으로 평가한 점수표를 새로운 컬럼 'score'에 대입
# word를 비교하여 평가를 수행, 점수는 누적합

In [87]:
score_dic[:2]

[{'word': '(-;', 'word_root': '(', 'polarity': '1'},
 {'word': '(;_;)', 'word_root': '(;_;)', 'polarity': '-1'}]

In [90]:
polarity_ = 0
for word, tag in df['classifier'][0] :
    for dic in score_dic :
        if dic['word'] == word :
            print(word, dic['polarity'])
            polarity_ += int(dic['polarity'])
polarity_, df['content'][0], df['star'][0]

많다 2
좋다 2
이상하다 -1
좋다 2


(5,
 '쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을 얼마나 많이했는데 방금 알바가입 머시했거든요? 10개 빨리주세요;; 진짜 나말고도 많더만 이정도면 네이버 이거 고객기만이지 장난하나;;;; 그리고 작가님들 다 좋거든요? 근데 왜 자꾸 하위권짜르세요 장난하나 작가님들 작품이 장난이에요? 이상한 만화는 안 짜르고 스토리 탄탄에 작화 좋은데 하위권 다 짤라;;;',
 1)

In [ ]:
# 형태소를 어떻게 처리했느냐, 매칭이 얼마나 되느냐, 사전의 표현을 얼마나 풍성하느냐에 따라
# 긍정과 부정이 계속 바뀐다
# -> 사람이 인위적으로 계속해서 사전을 추가하는 것은 사실상 불가능
# 매칭이 안 된 단어들은 유사도를 계산하여(벡터화 거리계산법) 가장 근접한 표현으로 찾아서 점수화하면 많은 보완이 될 것이고
# 이것이 사전에 반영되면 계산없이 바로 찾을 수도 있을 것이다
# 궁극적으로는 맥락이 더 중요하므로, 머신러닝이나 딥러닝으로 처리하는 게 더 효율적이다
# 챗봇, 번역 등도 머신러닝, 딥러닝 기반 구성

In [91]:
# 함수화
def scoreCalculator(x) :
    polarity_ = 0
    for word, tag in x :
        for dic in score_dic :
            if dic['word'] == word :
                polarity_ += int(dic['polarity'])
    return polarity_

In [92]:
df['score'] = df['classifier'].apply(scoreCalculator)

In [93]:
df.head(3)

,appname,content,good,name,regi,star,sentiment,classifier,score
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,4,솜담,2019-08-17,1,neg,"[(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)...",5
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,16,왈왈,2019-08-20,1,neg,"[(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ...",-4
2,네이버웹툰,"""개를 낳았다""를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸...",81,롄쀼,2019-08-14,5,pos,"[(개, Noun), (낳다, Verb), (를, Noun), (즐기다, Verb)...",0


In [94]:
df.loc[:, ['star', 'score']]

,star,score
0,1,5
1,1,-4
2,5,0
3,1,-1
4,1,0
5,5,5
6,2,1
7,3,0
8,1,-1
9,4,4
